# Advanced Mesh Generation: Domain with Observation Points

This notebook demonstrates more advanced GMSHFlow capabilities:

1. Creating domains with irregular shapes
2. Adding observation points with custom mesh sizes
3. Using mesh size fields for local refinement
4. Working with embedded geometries
5. Advanced visualization techniques

This example is closer to real-world groundwater modeling scenarios where you need fine mesh resolution around monitoring wells or other important features.

In [ ]:
# Import required libraries
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, Point, MultiPoint
import matplotlib.pyplot as plt
import gmshflow
from pathlib import Path

print(f"GMSHFlow version: {gmshflow.__version__}")
print("Libraries loaded successfully!")

## Step 1: Create an Irregular Domain

Let's create a more realistic, irregular domain shape that might represent a watershed or study area.

In [ ]:
# Create an irregular domain (more realistic shape)
np.random.seed(42)  # For reproducible results

# Create a base irregular polygon
theta = np.linspace(0, 2*np.pi, 20)
radius = 800 + 200 * np.sin(3*theta) + 100 * np.random.randn(20)
x_coords = 1000 + radius * np.cos(theta)
y_coords = 500 + 0.6 * radius * np.sin(theta)

# Close the polygon
domain_coords = list(zip(x_coords, y_coords)) + [(x_coords[0], y_coords[0])]

# Create the domain polygon
domain_polygon = Polygon(domain_coords)
domain_gdf = gpd.GeoDataFrame(
    {'name': ['irregular_domain']}, 
    geometry=[domain_polygon]
)

# Visualize the domain
fig, ax = plt.subplots(figsize=(10, 6))
domain_gdf.plot(ax=ax, alpha=0.7, edgecolor='black', color='lightblue')
ax.set_title('Irregular Domain Shape')
ax.set_xlabel('X coordinate (m)')
ax.set_ylabel('Y coordinate (m)')
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
plt.show()

print(f"Domain bounds: {domain_gdf.bounds}")
print(f"Domain area: {domain_gdf.geometry.area.iloc[0]:.0f} m²")

## Step 2: Create Observation Points

Now we'll add observation points (e.g., monitoring wells) that require fine mesh resolution around them.

In [ ]:
# Create observation points with different mesh size requirements
obs_points = [
    # High priority wells (fine mesh)
    Point(1000, 500),  # Center point
    Point(800, 400),   # Southwest
    Point(1200, 600),  # Northeast
    
    # Medium priority wells (medium mesh)
    Point(600, 500),   # West
    Point(1400, 500),  # East
    Point(1000, 300),  # South
    Point(1000, 700),  # North
    
    # Low priority wells (coarse mesh)
    Point(700, 650),   # Northwest
    Point(1300, 350),  # Southeast
]

# Assign mesh sizes based on priority
mesh_sizes = [
    # High priority: fine mesh (10m around wells)
    10, 10, 10,
    # Medium priority: medium mesh (25m around wells)
    25, 25, 25, 25,
    # Low priority: coarse mesh (50m around wells)
    50, 50
]

# Create GeoDataFrame for observation points
obs_gdf = gpd.GeoDataFrame({
    'well_id': [f'well_{i+1:02d}' for i in range(len(obs_points))],
    'priority': ['high']*3 + ['medium']*4 + ['low']*2,
    'cs': mesh_sizes  # cell size column for gmshflow
}, geometry=obs_points)

# Filter points to only include those inside the domain
obs_gdf = obs_gdf[obs_gdf.geometry.within(domain_polygon)]

print(f"Created {len(obs_gdf)} observation points inside the domain")
print(obs_gdf[['well_id', 'priority', 'cs']].head())

## Step 3: Visualize Domain and Observation Points

Let's create a visualization showing the domain with observation points colored by their mesh size requirements.

In [ ]:
# Create a comprehensive visualization
fig, ax = plt.subplots(figsize=(12, 8))

# Plot domain
domain_gdf.plot(ax=ax, alpha=0.3, edgecolor='black', color='lightgray', linewidth=2)

# Plot observation points with different colors for different priorities
priority_colors = {'high': 'red', 'medium': 'orange', 'low': 'blue'}
priority_sizes = {'high': 100, 'medium': 60, 'low': 30}

for priority in ['high', 'medium', 'low']:
    subset = obs_gdf[obs_gdf['priority'] == priority]
    if len(subset) > 0:
        subset.plot(
            ax=ax, 
            color=priority_colors[priority], 
            markersize=priority_sizes[priority],
            alpha=0.8,
            label=f'{priority.title()} priority (mesh size: {subset.cs.iloc[0]}m)'
        )

# Add well labels
for idx, row in obs_gdf.iterrows():
    ax.annotate(row['well_id'], 
                (row.geometry.x, row.geometry.y), 
                xytext=(5, 5), 
                textcoords='offset points',
                fontsize=8,
                ha='left')

ax.set_title('Domain with Observation Points\n(Different mesh size requirements)')
ax.set_xlabel('X coordinate (m)')
ax.set_ylabel('Y coordinate (m)')
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

## Step 4: Set Up Advanced Mesh Generation

Now we'll create a mesh with variable element sizes - fine around observation points and coarser elsewhere.

In [ ]:
# Define mesh parameters
domain_mesh_size = 100.0  # Background mesh size
model_name = "advanced_example"

# Initialize GMSH model and create domain
with gmshflow.GmshModel(model_name) as gmsh_model:
    print(f"GMSH model '{model_name}' initialized")
    
    # Create the main mesh domain
    mesh_domain = gmshflow.GmshMeshDomain(
        name="main_domain",
        gdf_dom=domain_gdf,
        cs_dom=domain_mesh_size,
    )
    
    print(f"Domain mesh size: {domain_mesh_size} m")
    print(f"Observation point mesh sizes: {obs_gdf.cs.tolist()} m")
    
    # Prepare the mesh domain
    mesh_domain.prepare_mesh_domain(
        mesh_area=10,      # Minimum area threshold for mesh elements
        min_overlap=0.1    # Minimum overlap threshold
    )
    print("Domain geometry prepared")
    
    # Create domain boundary and surface
    boundary_loop = mesh_domain.create_domain_loop_from_poly()
    surface_id = mesh_domain.create_domain_surface()
    print(f"Domain surface created with ID: {surface_id}")

## Step 5: Add Observation Points as Embedded Geometries

We'll embed the observation points into the mesh surface to ensure local refinement around them.

In [ ]:
with gmshflow.GmshModel(model_name) as gmsh_model:
    # Recreate domain setup
    mesh_domain = gmshflow.GmshMeshDomain(
        name="main_domain",
        gdf_dom=domain_gdf,
        cs_dom=domain_mesh_size,
    )
    
    mesh_domain.prepare_mesh_domain(mesh_area=10, min_overlap=0.1)
    boundary_loop = mesh_domain.create_domain_loop_from_poly()
    surface_id = mesh_domain.create_domain_surface()
    
    # Create observation points using PointGeometryHandler
    point_handler = gmshflow.PointGeometryHandler()
    point_handler.set_gdf_point(obs_gdf)
    
    # Create points in GMSH
    obs_point_ids = point_handler.create_point_from_point(df_coord=False)
    print(f"Created {len(obs_point_ids)} observation points in GMSH")
    
    # Embed observation points into the surface
    mesh_domain.add_embedded_points(
        id_point_list=obs_point_ids,
        surface_id=surface_id
    )
    print("Observation points embedded in mesh surface")
    
    # Generate mesh with variable sizing
    print("Generating mesh with variable element sizes...")
    gmsh_model.generate_mesh()
    print("Mesh generation completed!")
    
    # Get mesh statistics
    import gmsh
    node_tags, node_coords, _ = gmsh.model.mesh.getNodes()
    element_types, element_tags, _ = gmsh.model.mesh.getElements()
    
    num_nodes = len(node_tags)
    num_elements = sum(len(tags) for tags in element_tags)
    
    print(f"\nMesh statistics:")
    print(f"  - Number of nodes: {num_nodes}")
    print(f"  - Number of elements: {num_elements}")
    print(f"  - Background element size: {domain_mesh_size} m")
    print(f"  - Fine refinement around {len(obs_gdf)} observation points")

## Step 6: Export and Visualize Results

Export the mesh to Voronoi format and create detailed visualizations.

In [ ]:
# Create output directory
output_dir = Path("./output")
output_dir.mkdir(exist_ok=True)

with gmshflow.GmshModel(model_name) as gmsh_model:
    # Recreate the complete setup
    mesh_domain = gmshflow.GmshMeshDomain(
        name="main_domain",
        gdf_dom=domain_gdf,
        cs_dom=domain_mesh_size,
    )
    
    mesh_domain.prepare_mesh_domain(mesh_area=10, min_overlap=0.1)
    boundary_loop = mesh_domain.create_domain_loop_from_poly()
    surface_id = mesh_domain.create_domain_surface()
    
    # Add observation points
    point_handler = gmshflow.PointGeometryHandler()
    point_handler.set_gdf_point(obs_gdf)
    obs_point_ids = point_handler.create_point_from_point(df_coord=False)
    mesh_domain.add_embedded_points(obs_point_ids, surface_id)
    
    # Generate mesh
    gmsh_model.generate_mesh()
    
    # Export to Voronoi format
    output_name = "advanced_mesh_voronoi"
    surface_tags = [surface_id]
    
    print("Exporting to Voronoi format...")
    voronoi_gdf = mesh_domain.export_to_voronoi(
        ws=output_dir,
        name=output_name,
        surface_ids=surface_tags
    )
    
    print(f"Voronoi mesh exported to: {output_dir / output_name}.shp")
    print(f"Number of Voronoi cells: {len(voronoi_gdf)}")

## Step 7: Create Detailed Visualizations

Let's create comprehensive visualizations showing the mesh refinement around observation points.

In [ ]:
# Create detailed comparison plots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Original domain with observation points
domain_gdf.plot(ax=ax1, alpha=0.3, edgecolor='black', color='lightgray')
for priority in ['high', 'medium', 'low']:
    subset = obs_gdf[obs_gdf['priority'] == priority]
    if len(subset) > 0:
        subset.plot(ax=ax1, color=priority_colors[priority], 
                   markersize=priority_sizes[priority], alpha=0.8)
ax1.set_title('Domain with Observation Points')
ax1.set_xlabel('X coordinate (m)')
ax1.set_ylabel('Y coordinate (m)')
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Plot 2: Full Voronoi mesh
voronoi_gdf.plot(ax=ax2, alpha=0.7, edgecolor='blue', linewidth=0.3, color='lightgreen')
obs_gdf.plot(ax=ax2, color='red', markersize=20, alpha=0.8)
ax2.set_title(f'Complete Voronoi Mesh ({len(voronoi_gdf)} cells)')
ax2.set_xlabel('X coordinate (m)')
ax2.set_ylabel('Y coordinate (m)')
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')

# Plot 3: Zoomed view around high-priority wells
center_x, center_y = 1000, 500
zoom_extent = 200
zoom_bounds = [center_x - zoom_extent, center_x + zoom_extent, 
               center_y - zoom_extent, center_y + zoom_extent]

# Filter mesh and points for zoom area
zoom_mesh = voronoi_gdf.cx[zoom_bounds[0]:zoom_bounds[1], zoom_bounds[2]:zoom_bounds[3]]
zoom_obs = obs_gdf.cx[zoom_bounds[0]:zoom_bounds[1], zoom_bounds[2]:zoom_bounds[3]]

zoom_mesh.plot(ax=ax3, alpha=0.7, edgecolor='blue', linewidth=0.5, color='lightgreen')
zoom_obs.plot(ax=ax3, color='red', markersize=50, alpha=0.8)
ax3.set_xlim(zoom_bounds[0], zoom_bounds[1])
ax3.set_ylim(zoom_bounds[2], zoom_bounds[3])
ax3.set_title('Zoom: Fine Mesh Around High-Priority Wells')
ax3.set_xlabel('X coordinate (m)')
ax3.set_ylabel('Y coordinate (m)')
ax3.grid(True, alpha=0.3)
ax3.set_aspect('equal')

# Plot 4: Mesh size distribution (histogram)
if 'area' not in voronoi_gdf.columns:
    voronoi_gdf['area'] = voronoi_gdf.geometry.area

# Calculate equivalent cell sizes (assuming square cells)
voronoi_gdf['equiv_size'] = np.sqrt(voronoi_gdf['area'])

ax4.hist(voronoi_gdf['equiv_size'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax4.axvline(domain_mesh_size, color='red', linestyle='--', 
           label=f'Target size: {domain_mesh_size}m')
ax4.set_xlabel('Equivalent Cell Size (m)')
ax4.set_ylabel('Number of Cells')
ax4.set_title('Mesh Size Distribution')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Save the comprehensive figure
fig.savefig(output_dir / 'advanced_mesh_analysis.png', dpi=150, bbox_inches='tight')
print(f"Analysis plot saved to: {output_dir / 'advanced_mesh_analysis.png'}")

## Step 8: Mesh Quality Analysis

Let's analyze the mesh quality and refinement effectiveness.

In [ ]:
# Analyze mesh quality and refinement
print("=== MESH QUALITY ANALYSIS ===")
print(f"Total number of cells: {len(voronoi_gdf)}")
print(f"Total mesh area: {voronoi_gdf.geometry.area.sum():.0f} m²")
print(f"Domain area: {domain_gdf.geometry.area.iloc[0]:.0f} m²")
print(f"Area coverage: {voronoi_gdf.geometry.area.sum() / domain_gdf.geometry.area.iloc[0] * 100:.1f}%")

print("\n=== CELL SIZE STATISTICS ===")
size_stats = voronoi_gdf['equiv_size'].describe()
for stat, value in size_stats.items():
    print(f"{stat.capitalize()}: {value:.1f} m")

print("\n=== REFINEMENT ANALYSIS ===")
# Count cells in different size ranges
fine_cells = len(voronoi_gdf[voronoi_gdf['equiv_size'] < 30])
medium_cells = len(voronoi_gdf[(voronoi_gdf['equiv_size'] >= 30) & (voronoi_gdf['equiv_size'] < 60)])
coarse_cells = len(voronoi_gdf[voronoi_gdf['equiv_size'] >= 60])

print(f"Fine cells (<30m): {fine_cells} ({fine_cells/len(voronoi_gdf)*100:.1f}%)")
print(f"Medium cells (30-60m): {medium_cells} ({medium_cells/len(voronoi_gdf)*100:.1f}%)")
print(f"Coarse cells (>60m): {coarse_cells} ({coarse_cells/len(voronoi_gdf)*100:.1f}%)")

print("\n=== OBSERVATION POINT REFINEMENT ===")
# Check refinement around each observation point
for idx, obs in obs_gdf.iterrows():
    # Find cells within 100m of each observation point
    buffer = obs.geometry.buffer(100)
    nearby_cells = voronoi_gdf[voronoi_gdf.geometry.intersects(buffer)]
    
    if len(nearby_cells) > 0:
        avg_size = nearby_cells['equiv_size'].mean()
        min_size = nearby_cells['equiv_size'].min()
        print(f"{obs['well_id']} ({obs['priority']}): {len(nearby_cells)} nearby cells, "
              f"avg size: {avg_size:.1f}m, min size: {min_size:.1f}m")

## Summary

In this advanced example, we've demonstrated:

### Key Features Covered:
1. **Irregular Domain Creation**: Created a realistic, non-rectangular domain shape
2. **Multi-Priority Observation Points**: Added wells with different mesh size requirements
3. **Variable Mesh Sizing**: Achieved fine resolution around important features
4. **Embedded Geometries**: Used GMSHFlow's point embedding capabilities
5. **Advanced Visualization**: Created comprehensive plots showing mesh quality
6. **Quality Analysis**: Analyzed mesh statistics and refinement effectiveness

### Real-World Applications:
- **Groundwater Monitoring**: Fine mesh around monitoring wells
- **Contamination Studies**: High resolution near pollution sources
- **Aquifer Testing**: Detailed mesh for pumping test analysis
- **Environmental Modeling**: Variable resolution based on feature importance

### Key Takeaways:
- The `cs` (cell size) column in observation point GeoDataFrames controls local mesh refinement
- Embedded points automatically create local refinement without manual field definitions
- GMSHFlow handles the complex transition between fine and coarse mesh areas
- Mesh quality can be analyzed using cell area and equivalent size metrics

### Next Steps:
- Experiment with different mesh size ratios
- Add linear features (rivers, faults) using LineGeometryHandler
- Integrate with MODFLOW6 for groundwater flow simulation
- Try different domain shapes and observation point patterns
- Explore mesh optimization techniques for computational efficiency